# Tutorial 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/shubhvjain/coregtor/blob/release/docs/tutorial1.ipynb) 

In this tutorial we demonstrate the use of the `CoRegTor` tool to find transcription co regulators for a gene from gene expression data.

## Objective

The aim of this tutorial is to find potential co-regulators of the gene [GFAP](https://www.ncbi.nlm.nih.gov/gene/2670) by analyzing tissue gene expression data for Frontal cortex in adult brain. 

## Step 1 : Get data

Before we begin, let's gather all the data we require:
- Gene Expression data `ge_brain.gct`. This file contains tissue gene expression data for the Frontal Cortex (BA9) in an adult brain. The data is downloaded from the [GTEx portal](https://www.gtexportal.org/home/downloads/adult-gtex/bulk_tissue_expression)
- List of transcription factors `human_tf.txt` : This file was downloaded from [aertslab.org](https://resources.aertslab.org/cistarget/tf_lists/)



In [1]:
from pathlib import Path 
base_path = Path("docs/temp") # UPDATE THIS
data_file_path = Path(base_path/"brain_ge.gct") # UPDATE THIS 
tf_file_path = Path(base_path/"human_tf.txt") # UPDATE THIS
target_gene_name = "GFAP"


## Step 2 : Install and import the `CoRegTor` package

Using pip, `pip install coregtor` .

Or `poetry install coregtor` to add the package as a dependency in your project

In [2]:
# Install coregtor if not already installed, then import it
try:
    import coregtor
except ImportError:
    %pip install coregtor
    import coregtor

# Additional imports
from pathlib import Path


## Step 3 : Load gene expression data and transcription factors

Let's now load the data using the `read` method that accepts path to the gene expression data. The method outputs a pandas DataFrame with genes as columns and samples as rows. 

In [3]:
import pandas as pd
ge_data = coregtor.read(file_path=data_file_path)
tf_data = pd.read_csv(tf_file_path, names=["gene_name"], header=None)

In [4]:
ge_data

gene_name,DDX11L1,WASH7P,MIR6859-1,MIR1302-2HG,FAM138A,OR4G4P,OR4G11P,OR4F5,ENSG00000238009,CICP27,...,MT-ND4,MT-TH,MT-TS2,MT-TL2,MT-ND5,MT-ND6,MT-TE,MT-CYB,MT-TT,MT-TP
sample_name,,,,,,,,,,,,,,,,,,,,,
GTEX-1117F-0011-R10b-SM-GI4VE,0.000000,3.57928,0.0,0.093825,0.000000,0.000000,0.028731,0.046554,0.039501,0.058675,...,49762.2,1.177570,2.754330,0.000000,7311.39,4788.56,6.47666,28676.5,3.077750,1.19489
GTEX-111FC-0011-R10a-SM-GIN8G,0.000000,2.32926,0.0,0.025333,0.000000,0.052233,0.031030,0.016759,0.000000,0.031684,...,44692.0,0.953824,0.000000,1.544930,6831.00,5164.36,6.67677,26950.9,1.661970,3.54879
GTEX-117XS-0011-R10b-SM-GIN8Z,0.000000,4.79425,0.0,0.000000,0.046843,0.067977,0.020191,0.043622,0.013880,0.032987,...,39249.9,0.827551,0.967814,1.206360,5603.53,3585.51,6.20663,20794.9,0.432584,2.93902
GTEX-1192W-0011-R10b-SM-GHWOF,0.000000,3.83774,0.0,0.032159,0.045693,0.000000,0.039392,0.053189,0.013539,0.000000,...,50750.5,1.614480,2.832190,1.176750,9433.33,7697.90,12.51220,23405.4,1.265900,3.68601
GTEX-1192X-0011-R10a-SM-DO941,0.040388,1.47233,0.0,0.040318,0.000000,0.000000,0.049385,0.040010,0.050922,0.000000,...,31566.9,2.024070,0.591784,0.983528,4424.64,3568.41,4.55416,14051.5,0.529019,1.54038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTEX-ZVZQ-0011-R10b-SM-51MRT,0.017553,1.91964,0.0,0.070089,0.000000,0.180647,0.064389,0.092738,0.044262,0.043831,...,44939.6,3.078850,1.543150,2.137230,7019.94,6874.29,16.71380,24296.2,1.379490,2.23152
GTEX-ZXG5-0011-R10a-SM-57WDD,0.000000,1.07536,0.0,0.036646,0.000000,0.000000,0.044887,0.084853,0.000000,0.000000,...,62226.7,2.759570,1.613650,4.469730,11407.90,11061.80,15.17770,38732.2,1.442500,1.86677
GTEX-ZYFD-0011-R10a-SM-GPI91,0.000000,2.71020,0.0,0.000000,0.000000,0.000000,0.037432,0.000000,0.000000,0.030577,...,43740.3,0.000000,2.691290,0.745473,6574.39,5241.85,9.20498,24934.3,0.000000,1.55672


In [5]:
tf_data

,gene_name
0,ZNF354C
1,KLF12
2,ZNF143
3,ZIC2
4,ZNF274
...,...
1887,ZNF826P
1888,ZNF827
1889,ZNF831
1890,ZRSR2


## Step 4 : Create Ensemble model

Next, we generate a random forest ensemble model using the gene expression data to predict the expression value of the gene "GFAP" based on all other genes. 

Since we are interested in identifying potential transcription co regulators, we filter the data to include only transcription factors. We use the `create_model_input` function for filtering and preparing the input for training. It takes a dataframe  `t_factors` which should have a column `gene_name` listing the transcription factors to consider. The function outputs a tuple containing 2 data frames: X with the feature genes and Y with the target gene. These can then be passed to the `generate_model` function. 


In [6]:
# first generate the training input for the model
X,Y = coregtor.create_model_input(ge_data,target_gene_name,tf_data)
# use the training data to create a model
model = coregtor.create_model(X,Y,"rf",{ })

In [7]:
model

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## Step 5 : Generating tree paths 

The genes on the root node are important. These also serve as potential regulators in other tree based GRN inference methods. 

Forest based ensemble methods contains multiple decision tress. We want to analyze the structure of the trees in the model. 

For each tree, there exists multiple root to leaf paths. We first enumerate all the paths in all the trees in the model. 


In [8]:
all_paths = coregtor.tree_paths(model,X,Y)

In [9]:
all_paths

,tree,source,target,path_length,node1,node2,node3,node4,node5,node6,...,node9,node10,node11,node12,node13,node14,node15,node16,node17,node18
0,0,BBX,GFAP,8,YBX1,SP110,RFX4,TIGD4,NEUROG3,PAX2,...,None,None,None,None,None,None,None,None,None,None
1,0,BBX,GFAP,10,YBX1,SP110,VAX2,HMGB2,IRF9,DTL,...,CYCS,None,None,None,None,None,None,None,None,None
2,0,BBX,GFAP,10,YBX1,SP110,VAX2,HMGB2,IRF9,DTL,...,ATF5,None,None,None,None,None,None,None,None,None
3,0,BBX,GFAP,12,YBX1,SP110,VAX2,BARX2,PPP2R3B,ZFP41,...,IL24,SOX9,PAX6,None,None,None,None,None,None,None
4,0,BBX,GFAP,10,YBX1,SP110,RFX4,ZNF784,PRKAA2,ZHX2,...,BANP,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10972,99,HMBOX1,GFAP,10,HEY2,SP110,TAF1,SREBF1,DLX5,DDX43,...,ZFP82,None,None,None,None,None,None,None,None,None
10973,99,HMBOX1,GFAP,5,ZNF880,RBAK,SIX5,KLRG1,None,None,...,None,None,None,None,None,None,None,None,None,None
10974,99,HMBOX1,GFAP,10,HEY2,SP110,TAF1,SREBF1,TGIF2,ZSCAN5C,...,ZNF490,None,None,None,None,None,None,None,None,None
10975,99,HMBOX1,GFAP,11,HEY2,SP110,TAF1,SREBF1,DLX5,DDX43,...,ZFP82,ZNF565,None,None,None,None,None,None,None,None


## Step 6 : Generating context for all root nodes 

In the table of paths above, we observe many unique genes appear as root nodes. Since we train the decision trees to predict the same target gene, the leaf nodes for all paths are the same. 

We can thus consider these paths as potential regulatory links, where genes at the root regulates the target. All the genes at the root become potential regulators or the target. However, note that  there are multiple intermediate nodes between the root and the target. Comparing these 


We consider the root nodes as potential regulators of the target gene and to find if they are co regulators, we compare how similar the intermediate nodes are in between 2 unique root nodes. 




In [10]:
pathset = coregtor.create_context(all_paths)

In [18]:
pathset.keys()

dict_keys(['BBX', 'HIRIP3', 'NKX6-2', 'ATOH8', 'ZNF577', 'ZNF846', 'HMBOX1', 'MAFK', 'ZFP69', 'IRF9', 'TRMT1', 'DPRX', 'ZNF652', 'GLI4', 'YBX1', 'CREB3L4', 'ZBTB48', 'SP110', 'TBX19', 'ZNF326', 'BCL6', 'ZNF471', 'ALX3', 'ZNF143', 'ZNF485', 'ZFP57', 'ZNF114', 'RFX4', 'ID1', 'SFT2D1', 'ZNF18', 'ZNF536', 'JRKL', 'HCLS1', 'EEF1D', 'ZSCAN16', 'ZNF578', 'HTATIP2', 'HSF4', 'FOXO4', 'PPP2R3B', 'NFYA', 'HNRNPUL1', 'KLF15'])

## Step 7 : Comparing context of all root nodes with each other

similarity 

In [13]:
# transforming the context into a more comparable representation 
gf_histogram = coregtor.transform_context(pathset,method="gene_frequency")

In [14]:
gf_histogram

,SP110,RFX4,TIGD4,NEUROG3,PAX2,VAX2,HMGB2,IRF9,DTL,ZNF460,...,LRRFIP1,ADNP2,HIVEP1,ATF7,ZNF766,PRNP,ZNF735,PDX1,GATAD2A,ZNF324B
BBX,73,29,22,9,12,44,17,14,10,7,...,0,0,0,0,0,0,0,0,0,0
HIRIP3,0,0,0,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NKX6-2,0,0,0,0,0,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
ATOH8,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZNF577,70,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
ZNF846,0,0,0,0,0,0,6,23,9,0,...,0,0,0,0,0,0,0,0,0,0
HMBOX1,318,5,3,1,5,8,1,175,1,0,...,0,0,0,0,0,0,0,0,0,0
MAFK,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZFP69,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
IRF9,137,0,1,5,1,11,3,0,6,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
sim_matrix = coregtor.compare_context(gf_histogram,"cosine")

In [17]:
sim_matrix

,BBX,HIRIP3,NKX6-2,ATOH8,ZNF577,ZNF846,HMBOX1,MAFK,ZFP69,IRF9,...,EEF1D,ZSCAN16,ZNF578,HTATIP2,HSF4,FOXO4,PPP2R3B,NFYA,HNRNPUL1,KLF15
BBX,1.000000,0.013154,0.034314,0.026428,0.350282,0.038503,0.344944,0.307093,0.006496,0.311204,...,0.493064,0.004241,0.026826,0.023906,0.021723,0.330247,0.438607,0.038793,0.039063,0.001498
HIRIP3,0.013154,1.000000,0.011029,0.022970,0.029681,0.015394,0.142369,0.149039,0.193309,0.286811,...,0.047937,0.010904,0.008858,0.048731,0.079219,0.017788,0.127633,0.038334,0.112439,0.127621
NKX6-2,0.034314,0.011029,1.000000,0.050475,0.019280,0.284606,0.394946,0.017250,0.311219,0.122385,...,0.005607,0.300193,0.523954,0.393550,0.412418,0.019055,0.012243,0.003264,0.036243,0.323032
ATOH8,0.026428,0.022970,0.050475,1.000000,0.007474,0.036283,0.049953,0.012301,0.017746,0.136366,...,0.015632,0.069195,0.003922,0.002072,0.002949,0.009816,0.008670,0.108131,0.008399,0.012755
ZNF577,0.350282,0.029681,0.019280,0.007474,1.000000,0.251343,0.274627,0.396628,0.306698,0.346569,...,0.292039,0.013430,0.005452,0.006402,0.008178,0.616324,0.328666,0.331659,0.281406,0.015077
ZNF846,0.038503,0.015394,0.284606,0.036283,0.251343,1.000000,0.345924,0.155773,0.570722,0.265057,...,0.014145,0.277339,0.182778,0.240215,0.283613,0.264202,0.186912,0.296992,0.260537,0.250049
HMBOX1,0.344944,0.142369,0.394946,0.049953,0.274627,0.345924,1.000000,0.341979,0.346151,0.434927,...,0.292037,0.417910,0.366341,0.431302,0.443863,0.249911,0.355517,0.132789,0.196643,0.344010
MAFK,0.307093,0.149039,0.017250,0.012301,0.396628,0.155773,0.341979,1.000000,0.197121,0.350308,...,0.301569,0.371011,0.006187,0.353739,0.134720,0.402244,0.304858,0.149975,0.458870,0.044637
ZFP69,0.006496,0.193309,0.311219,0.017746,0.306698,0.570722,0.346151,0.197121,1.000000,0.303834,...,0.011648,0.314612,0.227373,0.271071,0.360290,0.341019,0.019788,0.314607,0.269368,0.347570
IRF9,0.311204,0.286811,0.122385,0.136366,0.346569,0.265057,0.434927,0.350308,0.303834,1.000000,...,0.271254,0.148667,0.102509,0.178363,0.138231,0.371597,0.315959,0.197280,0.175219,0.160146


## Step 3-7 all at once


In [16]:
#

## Step 8 : Interactive generation of co-regulating gene clusters

# Validation of results 